In [10]:
import pandas as pd
snapp_data = pd.read_csv("/content/Snappfood - Sentiment Analysis.csv", sep="\t", encoding="utf-8")
snapp_data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [2]:
!pip install nltk hazm

In [8]:
import requests

url = "https://raw.githubusercontent.com/kharazi/persian-stopwords/master/persian"
response = requests.get(url)
persian_stop_words = response.text.split("\n")


In [12]:
import pandas as pd
from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer
import re
import string

# 1. Initialize tools
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

# 2. Load stopwords
# stopwords = set(open("https://raw.githubusercontent.com/ashkan-software/persian-stopwords/master/stopwords.txt").read().split())

# 3. Define cleaning function
def clean_persian_text(text):
    # Normalize
    text = normalizer.normalize(str(text))

    # Remove emojis, URLs, and English
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[a-zA-Z]", "", text)
    text = re.sub(r"[^\w\s\u0600-\u06FF]", "", text)  # keep Persian letters

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    filtered = [word for word in tokens if word not in persian_stop_words and len(word) > 2]

    # Optional: Stemming or Lemmatizing
    # stemmed = [stemmer.stem(word) for word in filtered]
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered]  # Alternative

    return " ".join(lemmatized)

# 4. Apply to dataset
snapp_data['clean_text'] = snapp_data['comment'].apply(clean_persian_text)

# Preview result
snapp_data[['comment', 'clean_text']].head()


,comment,clean_text
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,حیف وقت نوشت#نویس سرویس دهیتون افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,قرار ساعته برسه نیم ساعت موقع دید#بین پلاک خفن...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,قیمت مدل کیفیت سازگاری نداره ظاهر فریبنده پرش ...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,عالی اندازه کیفیت امیداورم کیفیتتون باشه مشتری...
4,شیرینی وانیلی فقط یک مدل بود.,شیرینی وانیل مدل


In [16]:
snapp_data.to_csv("Snapp_clean.csv", sep='\t', encoding='utf-8')